# Radiative transfer

To test whether the radiative transfer calculation is working correctly, I will examine a model with erroneous intensity calculations (with some sightlines being $~10^{40}K$). Below we can see how the whole sky map can be narrowed to just a few erroneous sightlines.

There is also an option when evaluating the refined sightlines to save the intermediate arrays in numpy files (`I.npy`, `e.npy`, `de.npy`, `k.npy`, `dk.npy`, `a.npy`, and `b.npy`, corresponding to $I_\nu$, $\epsilon_\nu$, $\Delta \epsilon_\nu$, $\kappa_\nu$, $\Delta \kappa_\nu$, $A_\nu$, and $B_\nu$, respectively) to load and work with the data.

In [6]:
from pprint import pprint

import numpy as np
from astropy.io import fits

In [3]:
chmap = fits.open(r"c:\users\cyani\projects\pdr\KT3_history\MilkyWay\r400_cm1-1_d1_uv10\channel_intensity.fits")

In [5]:
chmap[1].shape

(701, 5, 180, 18)

In [7]:
pprint(chmap[1].header)

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -64 / array data type                                
NAXIS   =                    4 / number of array dimensions                     
NAXIS1  =                   18                                                  
NAXIS2  =                  180                                                  
NAXIS3  =                    5                                                  
NAXIS4  =                  701                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
TYPE    = 'Species transitions'                                                 
BUNIT   = 'K       '                                                            
CTYPE1  = 'Wavelength'                                                          
CUNIT1  = 'm       '        

In [4]:
np.where(chmap[1].data[:,:,:,0]>10**30)

(array([341, 341, 341, 341, 359, 359, 359, 359], dtype=int64),
 array([2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([170, 171, 172, 173,   6,   7,   8,   9], dtype=int64))

In [9]:
lon = np.linspace(chmap[1].header['CRVAL2'] - chmap[1].header['CDELT2']*chmap[1].header['CRPIX2'],
                  chmap[1].header['CRVAL2'] + chmap[1].header['CDELT2']*chmap[1].header['CRPIX2'],
                  num=chmap[1].header['NAXIS2'])

In [11]:
lon[170],lon[173]

(2.8414642208742613, 2.9473572974285815)

### Theory

In order to test the correct calculation of radiative transfer, the theory must first be covered. The radiative transfer (RT) equation,

$$
dI_\nu = - I_\nu \kappa_\nu ds + \epsilon_\nu ds
$$

must be integrated to calculate the observed intensity. Simple applications of RT assume constant $\epsilon_\nu$ and $\kappa_\nu$ between voxels to obtain the observed intensity. The benefit of `kosmatau3d` is the more accurate calculation of the RT equation. Here the emissivity and absorption terms, $\epsilon_\nu$ and $\kappa_\nu$, are still used, but the first-order linear terms, $\Delta\epsilon_\nu = \frac{\epsilon_\nu}{\Delta s}$ and $\Delta\kappa_\nu = \frac{\kappa_\nu}{\Delta s}$, also appear. Here $\Delta s$ is the width of each voxel. This allows us to define specific regimes for the way we integrate the RT equation:

 - *Basic*: $\Delta\kappa = 0$ and $| \kappa \Delta s | < 10^{-10}$
 - *Simple*: $\kappa > 10^3 \Delta \kappa \Delta s$
 - *Complex*: otherwise
   - *method 1*: $\Delta \kappa > 0$
   - *method 2*: $\Delta \kappa < 0$
   
The complex methods of calculating the RT equation are the most accurate, and should be used by default. The other methods are simplifications that can be made. The *Basic* regime occurs when there is (mostly) no absorption. Since $\kappa_\nu = 0$ and $\Delta\kappa_\nu = 0$, the RT equation becomes, 

$$
\int_0^{I_\nu} dI_\nu = \int_0^{\Delta s} \left( \epsilon_\nu + \Delta\epsilon_\nu s' \right) ds', \\
I_\nu = \epsilon_\nu \Delta s + \frac{1}{2} \Delta\epsilon_\nu \left( \Delta s \right)^2 + I_{\nu,0}.
$$

The *Simple* regime occurs when there is constant absorption, so $\Delta\kappa_\nu = 0$ and $\kappa_\nu \neq 0$. This has a more-complex equation:

$$
\int_0^{I_\nu} dI_\nu = e^{-\kappa_\nu \Delta s} \left( \int_0^{\Delta s} (\epsilon_\nu + \Delta \epsilon_\nu s') e^{\kappa_\nu s'} ds' + I_0 \right), \\
I_\nu = \left( \frac{\epsilon_\nu \kappa_\nu + \Delta\epsilon_\nu (\kappa_\nu \Delta s - 1)}{\kappa_\nu^2} \right) - e^{-\kappa_\nu \Delta s} \left( \frac{\epsilon_\nu \kappa_\nu - \Delta\epsilon_\nu}{\kappa_\nu^2} \right) + e^{-\kappa_\nu \Delta s} I_{\nu,0}.
$$

Finally, the *Complex* integration is the full unsimplified integration of the RT equation. The only distinction made is if $\Delta\kappa \lessgtr 0$.

$$
I_\nu = \frac{\Delta\epsilon_\nu}{\Delta\kappa_\nu} \left( 1 - e^{-\kappa_\nu \Delta s - \frac{1}{2} \Delta\kappa_\nu (\Delta s)^2} \right) - \frac{\epsilon_\nu \Delta\kappa_\nu - \kappa_\nu \Delta\epsilon_\nu}{\Delta\kappa_\nu} \sqrt{\frac{\pi}{2|\Delta\kappa_\nu|}} \left( e^{a^2 - b^2}\tilde{E}(a) - \tilde{E}(b) \right) + I_{\nu,0} e^{-\kappa_\nu \Delta s - \frac{1}{2} \Delta\kappa_\nu (\Delta s)^2}
$$

Here $a=\frac{\kappa_\nu}{\sqrt{2\Delta\kappa_\nu}}$ and $b=\frac{\kappa_\nu + \Delta\kappa_\nu \Delta s}{\sqrt{2 \Delta\kappa_\nu}}$, while $\tilde{E}$ is an approximation of the imaginary error function. The distinction we make is how we treat the error function if $a,b$ are real or imaginary values.

### Calculation

With the radiative transfer method focused on the offending sightlines, we can obtain these values used in the RT equation. Now it is simply a matter of re-evaluating manually to compare the results (and find any relevant error). Since our model of interest has voxels with size $s=400pc$, we have $\Delta s = s [cm]$.

In [1]:
# CO 3-2
intensity = 1.0620254730090504e+39


epsilon = '[1.26730033e-24 1.26730033e-24 1.26322704e-24 1.26322704e-24 1.09893712e-24 1.09893712e-24 1.09497296e-24 1.09497296e-24 1.08449571e-24 1.08449571e-24 9.53655205e-25 9.53655205e-25 9.50514402e-25 9.50514402e-25 9.41470783e-25 9.41470783e-25 8.25153994e-25 8.25153994e-25 8.22513949e-25 8.22513949e-25 9.26886877e-25 9.26886877e-25 8.14673483e-25 8.14673483e-25 7.12406842e-25 7.12406842e-25 8.01810429e-25 8.01810429e-25 7.10464418e-25 7.10464418e-25 7.04641650e-25 7.04641650e-25 7.93700191e-25 7.93700191e-25 6.95028883e-25 6.95028883e-25 6.22814504e-25 6.22814504e-25 6.21171729e-25 6.21171729e-25 6.16273518e-25 6.16273518e-25 1.79045484e-24 1.79045484e-24 6.08168701e-25 6.08168701e-25 5.40932430e-25 5.40932430e-25 5.39554042e-25 5.39554042e-25 3.00302544e-24 3.00302544e-24 5.35388989e-25 5.35388989e-25 5.28512306e-25 5.28512306e-25 4.72274506e-25 4.72274506e-25 4.71285183e-25 4.71285183e-25 3.13530229e-24 3.13530229e-24 4.68348705e-25 4.68348705e-25 4.63479202e-25 4.63479202e-25 2.94460334e-23 2.94460334e-23 2.65227705e-24 2.65227705e-24 4.16907151e-25 4.16907151e-25 4.14096661e-25 4.14096661e-25 2.73744654e-23 2.73744654e-23 4.09433997e-25 4.09433997e-25 2.01303156e-24 2.01303156e-24 3.63159423e-25 3.63159423e-25 3.60513597e-25 3.60513597e-25 2.29521851e-23 2.29521851e-23 3.56104692e-25 3.56104692e-25 1.36847186e-24 1.36847186e-24 7.12126821e-23 7.12126821e-23 3.10662500e-25 3.10662500e-25 3.08145848e-25 3.08145848e-25 1.76491170e-23 1.76491170e-23 3.03974164e-25 3.03974164e-25 6.04206288e-23 6.04206288e-23 8.32520052e-25 8.32520052e-25 2.59162173e-25 2.59162173e-25 1.28825051e-23 1.28825051e-23 2.57040966e-25 2.57040966e-25 2.53560724e-25 2.53560724e-25 4.95338018e-23 4.95338018e-23 4.00353363e-25 4.00353363e-25 8.41028483e-24 8.41028483e-24 2.16757840e-25 2.16757840e-25 2.14414910e-25 2.14414910e-25 3.82477297e-23 3.82477297e-23 2.30304879e-25 2.30304879e-25 5.63645704e-24 5.63645704e-24 1.86037442e-25 1.86037442e-25 1.83800040e-25 1.83800040e-25 3.05467034e-23 3.05467034e-23 1.83558131e-25 1.83558131e-25 3.97937057e-23 3.97937057e-23 4.06949091e-24 4.06949091e-24 1.57330371e-25 1.57330371e-25 2.44611898e-23 2.44611898e-23 1.55546397e-25 1.55546397e-25 1.53119886e-25 1.53119886e-25 3.58954568e-23 3.58954568e-23 2.49081704e-24 2.49081704e-24 1.33599755e-25 1.33599755e-25 1.78060455e-23 1.78060455e-23 1.32018965e-25 1.32018965e-25 1.29814080e-25 1.29814080e-25 3.25900593e-23 3.25900593e-23 1.43210268e-24 1.43210268e-24 1.22874824e-23 1.22874824e-23 1.09756164e-25 1.09756164e-25 1.07902949e-25 1.07902949e-25 2.83338010e-23 2.83338010e-23 8.19947876e-25 8.19947876e-25 1.83553071e-23 1.83553071e-23 8.22252312e-24 8.22252312e-24 9.25261473e-26 9.25261473e-26 2.34942059e-23 2.34942059e-23 9.12997801e-26 9.12997801e-26 4.68149856e-25 4.68149856e-25 1.90552898e-23 1.90552898e-23 5.38333326e-24 5.38333326e-24 7.92942875e-26 7.92942875e-26 1.88285567e-23 1.88285567e-23 7.81192846e-26 7.81192846e-26 2.24163879e-25 2.24163879e-25 1.92269921e-23 1.92269921e-23 3.38578085e-24 3.38578085e-24 6.71658467e-26 6.71658467e-26 1.44140066e-23 1.44140066e-23 6.62451041e-26 6.62451041e-26 1.02983983e-25 1.02983983e-25 1.82001231e-23 1.82001231e-23 2.22149664e-24 2.22149664e-24 7.32442476e-24 7.32442476e-24 1.11057846e-23 1.11057846e-23 5.66820613e-26 5.66820613e-26 1.66068858e-23 1.66068858e-23 6.40956219e-26 6.40956219e-26 1.55174856e-24 1.55174856e-24 7.80387542e-24 7.80387542e-24 8.66528558e-24 8.66528558e-24 4.77988045e-26 4.77988045e-26 1.48255449e-23 1.48255449e-23 4.73447265e-26 4.73447265e-26 9.12088828e-25 9.12088828e-25 8.31200706e-24 8.31200706e-24 5.88810921e-24 5.88810921e-24 4.02127833e-26 4.02127833e-26 1.27323717e-23 1.27323717e-23 3.95018158e-26 3.95018158e-26 5.07495086e-25 5.07495086e-25 9.56943766e-24 9.56943766e-24 3.97758886e-24 3.97758886e-24 1.07205486e-23 1.07205486e-23 3.27192500e-26 3.27192500e-26 9.79346627e-24 9.79346627e-24 3.55737713e-25 3.55737713e-25 3.10318324e-24 3.10318324e-24 3.17902538e-24 3.17902538e-24 9.31854594e-24 9.31854594e-24 9.36604680e-24 9.36604680e-24]'


depsilon = '[ 0.00000000e+00 -3.55953745e-47  0.00000000e+00 -1.43568425e-45  0.00000000e+00 -3.46416889e-47  0.00000000e+00 -9.15578209e-47  0.00000000e+00 -1.14337900e-45  0.00000000e+00 -2.74466049e-47  0.00000000e+00 -7.90296908e-47  0.00000000e+00 -1.01646029e-45  0.00000000e+00 -2.30706244e-47  0.00000000e+00  9.12086194e-46  0.00000000e+00 -9.80601858e-46  0.00000000e+00 -8.93679931e-46  0.00000000e+00  7.81273254e-46  0.00000000e+00 -7.98247560e-46  0.00000000e+00 -5.08835695e-47  0.00000000e+00  7.78258000e-46  0.00000000e+00 -8.62261316e-46  0.00000000e+00 -6.31061513e-46  0.00000000e+00 -1.43557561e-47  0.00000000e+00 -4.28041060e-47  0.00000000e+00  1.02608463e-44  0.00000000e+00 -1.03316720e-44  0.00000000e+00 -5.87559205e-46  0.00000000e+00 -1.20453454e-47  0.00000000e+00  2.15275961e-44  0.00000000e+00 -2.15639934e-44  0.00000000e+00 -6.00934286e-47  0.00000000e+00 -4.91446599e-46  0.00000000e+00 -8.64542040e-48  0.00000000e+00  2.32801097e-44  0.00000000e+00 -2.33057708e-44  0.00000000e+00 -4.25532462e-47  0.00000000e+00  2.53270536e-43  0.00000000e+00 -2.34143236e-43  0.00000000e+00 -1.95342801e-44  0.00000000e+00 -2.45601031e-47  0.00000000e+00  2.35599216e-43  0.00000000e+00 -2.35639962e-43  0.00000000e+00  1.40133962e-44  0.00000000e+00 -1.44177769e-44  0.00000000e+00 -2.31211427e-47  0.00000000e+00  1.97422378e-43  0.00000000e+00 -1.97460906e-43  0.00000000e+00  8.84679710e-45  0.00000000e+00  6.10349263e-43  0.00000000e+00 -6.19593167e-43  0.00000000e+00 -2.19923305e-47  0.00000000e+00  1.51537958e-43  0.00000000e+00 -1.51574413e-43  0.00000000e+00  5.25342841e-43  0.00000000e+00 -5.20724025e-43  0.00000000e+00 -5.01041615e-45  0.00000000e+00  1.10311907e-43  0.00000000e+00 -1.10330444e-43  0.00000000e+00 -3.04128762e-47  0.00000000e+00  4.30646416e-43  0.00000000e+00 -4.29363636e-43  0.00000000e+00  6.99965788e-44  0.00000000e+00 -7.16009694e-44  0.00000000e+00 -2.04742220e-47  0.00000000e+00  3.32362634e-43  0.00000000e+00 -3.32223776e-43  0.00000000e+00  4.72428706e-44  0.00000000e+00 -4.76297115e-44  0.00000000e+00 -1.95520370e-47  0.00000000e+00  2.65333026e-43  0.00000000e+00 -2.65335140e-43  0.00000000e+00  3.46142140e-43  0.00000000e+00 -3.12184046e-43  0.00000000e+00 -3.41872904e-44  0.00000000e+00  2.12384715e-43  0.00000000e+00 -2.12400305e-43  0.00000000e+00 -2.12046130e-47  0.00000000e+00  3.12342411e-43  0.00000000e+00 -2.91913921e-43  0.00000000e+00 -2.05990708e-44  0.00000000e+00  1.54434622e-43  0.00000000e+00 -1.54448436e-43  0.00000000e+00 -1.92678759e-47  0.00000000e+00  2.83661118e-43  0.00000000e+00 -2.72280777e-43  0.00000000e+00  9.48621648e-44  0.00000000e+00 -1.06417786e-43  0.00000000e+00 -1.61947406e-47  0.00000000e+00  2.46658327e-43  0.00000000e+00 -2.40435963e-43  0.00000000e+00  1.53236661e-43  0.00000000e+00 -8.85476009e-44  0.00000000e+00 -7.10457981e-44  0.00000000e+00  2.04500816e-43  0.00000000e+00 -2.04511533e-43  0.00000000e+00  3.29318873e-45  0.00000000e+00  1.62427882e-43  0.00000000e+00 -1.19475452e-43  0.00000000e+00 -4.63505316e-44  0.00000000e+00  1.63844626e-43  0.00000000e+00 -1.63854894e-43  0.00000000e+00  1.27624338e-45  0.00000000e+00  1.66060467e-43  0.00000000e+00 -1.38431970e-43  0.00000000e+00 -2.90004599e-44  0.00000000e+00  1.25373079e-43  0.00000000e+00 -1.25381125e-43  0.00000000e+00  3.21050919e-46  0.00000000e+00  1.58145900e-43  0.00000000e+00 -1.39632803e-43  0.00000000e+00  4.45930795e-44  0.00000000e+00  3.30442589e-44  0.00000000e+00 -9.65550550e-44  0.00000000e+00  1.44627659e-43  0.00000000e+00 -1.44562874e-43  0.00000000e+00  1.30001881e-44  0.00000000e+00  5.46356099e-44  0.00000000e+00  7.52762550e-45  0.00000000e+00 -7.53058369e-44  0.00000000e+00  1.29138642e-43  0.00000000e+00 -1.29142610e-43  0.00000000e+00  7.55675987e-45  0.00000000e+00  6.46658417e-44  0.00000000e+00 -2.11817739e-44 0.00000000e+00 -5.11031517e-44  0.00000000e+00  1.10913271e-43  0.00000000e+00 -1.10919484e-43  0.00000000e+00  4.08966393e-45  0.00000000e+00  7.91898133e-44  0.00000000e+00 -4.88656223e-44  0.00000000e+00  5.89248626e-44  0.00000000e+00 -9.33979885e-44  0.00000000e+00  8.52964724e-44  0.00000000e+00 -8.24737033e-44  0.00000000e+00  2.40091676e-44  0.00000000e+00  6.62763548e-46  0.00000000e+00  5.36515745e-44  0.00000000e+00  4.15096886e-46  0.00000000e+00]'


kappa = '[-5.41042364e-27 -5.41042364e-27 -2.15649140e-27 -2.15649140e-27  7.46792444e-27  7.46792444e-27 -4.98079978e-28 -4.98079978e-28 -4.49455046e-27 -4.49455046e-27  8.73483438e-27  8.73483438e-27  5.19764069e-27  5.19764069e-27 -6.06365838e-29 -6.06365838e-29  8.23370750e-27  8.23370750e-27  6.75261809e-27  6.75261809e-27 -2.31289051e-27 -2.31289051e-27  3.44438584e-27  3.44438584e-27  7.46849673e-27  7.46849673e-27  6.80643060e-28  6.80643060e-28  6.81898295e-27  6.81898295e-27  4.72404346e-27  4.72404346e-27 -3.98101267e-26 -3.98101267e-26  2.37840713e-27  2.37840713e-27  6.14984723e-27  6.14984723e-27  5.55410866e-27  5.55410866e-27  4.28373950e-27  4.28373950e-27  2.15707499e-25  2.15707499e-25 -2.67346781e-26 -2.67346781e-26  4.88804926e-27  4.88804926e-27  4.88133439e-27  4.88133439e-27  5.25272609e-25  5.25272609e-25  3.79719635e-27  3.79719635e-27 -2.18859929e-26 -2.18859929e-26  4.70937075e-27  4.70937075e-27  4.26052949e-27  4.26052949e-27  5.82670089e-25  5.82670089e-25  3.80581354e-27  3.80581354e-27 -1.83464155e-26 -1.83464155e-26  6.69216855e-24  6.69216855e-24  4.92843526e-25  4.92843526e-25  4.51703374e-27  4.51703374e-27  4.12541246e-27  4.12541246e-27  6.26902380e-24  6.26902380e-24  3.73477570e-27  3.73477570e-27  3.62206315e-25  3.62206315e-25  4.47365126e-27  4.47365126e-27  4.50043591e-27  4.50043591e-27  5.27517273e-24  5.27517273e-24  4.19985015e-27  4.19985015e-27  2.27959973e-25  2.27959973e-25  1.65960373e-23  1.65960373e-23  5.02675121e-27  5.02675121e-27  4.77198197e-27  4.77198197e-27  4.05980592e-24  4.05980592e-24  4.81743854e-27  4.81743854e-27  1.41195455e-23  1.41195455e-23  1.18258121e-25  1.18258121e-25  5.19336397e-27  5.19336397e-27  2.95733981e-24  2.95733981e-24  5.20376824e-27  5.20376824e-27  5.00334784e-27  5.00334784e-27  1.15830414e-23  1.15830414e-23  3.17005243e-26  3.17005243e-26  1.91915368e-24  1.91915368e-24  5.27608681e-27  5.27608681e-27  5.10237429e-27  5.10237429e-27  8.94142164e-24  8.94142164e-24  2.45007346e-27  2.45007346e-27  1.27533267e-24  1.27533267e-24  5.01615665e-27  5.01615665e-27  5.00564380e-27  5.00564380e-27  7.13277745e-24  7.13277745e-24 -8.27020221e-29 -8.27020221e-29  9.30493953e-24  9.30493953e-24  9.13745831e-25  9.13745831e-25  4.81677814e-27  4.81677814e-27  5.70141248e-24  5.70141248e-24  4.79816244e-27  4.79816244e-27  5.04569430e-31  5.04569430e-31  8.38251976e-24  8.38251976e-24  5.48908005e-25  5.48908005e-25  4.55681941e-27  4.55681941e-27  4.13678493e-24  4.13678493e-24  4.44819555e-27  4.44819555e-27  6.15495904e-28  6.15495904e-28  7.59509388e-24  7.59509388e-24  3.06401660e-25  3.06401660e-25  2.84889881e-24  2.84889881e-24  4.18028373e-27  4.18028373e-27  1.10945140e-27  1.10945140e-27  6.60560659e-24  6.60560659e-24  1.68408284e-25  1.68408284e-25  4.27164855e-24  4.27164855e-24  1.90302989e-24  1.90302989e-24  3.89003835e-27  3.89003835e-27  5.47931814e-24  5.47931814e-24  3.86388854e-27  3.86388854e-27  8.98899244e-26  8.98899244e-26  4.44070516e-24  4.44070516e-24  1.24037534e-24  1.24037534e-24  3.59081773e-27  3.59081773e-27  4.38397724e-24  4.38397724e-24  3.51409922e-27  3.51409922e-27  3.59998356e-26  3.59998356e-26  4.47584978e-24  4.47584978e-24  7.74675535e-25  7.74675535e-25  3.26108740e-27  3.26108740e-27  3.34830264e-24  3.34830264e-24  3.24013994e-27  3.24013994e-27  1.05863649e-26  1.05863649e-26  4.23053253e-24  4.23053253e-24  5.04860333e-25  5.04860333e-25  1.69407815e-24  1.69407815e-24  2.57554790e-24  2.57554790e-24  3.01501944e-27  3.01501944e-27  3.85691897e-24  3.85691897e-24  3.80705035e-27  3.80705035e-27  3.50541383e-25  3.50541383e-25  1.80629070e-24  1.80629070e-24  2.00625341e-24  2.00625341e-24  2.75993289e-27  2.75993289e-27  3.43848966e-24  3.43848966e-24  1.96995246e-27  1.96995246e-27  2.02861832e-25  2.02861832e-25  1.92278032e-24  1.92278032e-24  1.35629191e-24  1.35629191e-24  2.50828022e-27  2.50828022e-27  2.93892820e-24  2.93892820e-24  1.80251202e-27  1.80251202e-27  1.09681062e-25  1.09681062e-25  2.20381815e-24  2.20381815e-24  9.06205459e-25  9.06205459e-25  2.44696059e-24  2.44696059e-24  1.72466971e-27  1.72466971e-27  2.22678422e-24  2.22678422e-24  7.46330311e-26  7.46330311e-26  6.89963543e-25  6.89963543e-25  7.15428095e-25  7.15428095e-25  2.07144108e-24  2.07144108e-24  2.06857724e-24  2.06857724e-24]'


dkappa = '[ 0.00000000e+00  2.84352151e-47  0.00000000e+00  8.41051119e-47  0.00000000e+00 -6.96127125e-47  0.00000000e+00 -3.49240517e-47  0.00000000e+00  1.15607940e-46  0.00000000e+00 -3.09105587e-47  0.00000000e+00 -4.59506328e-47  0.00000000e+00  7.24819821e-47  0.00000000e+00 -1.29428313e-47  0.00000000e+00 -7.92209758e-47  0.00000000e+00  5.03112479e-47  0.00000000e+00  3.51655937e-47  0.00000000e+00 -5.93171785e-47  0.00000000e+00  5.36412570e-47  0.00000000e+00 -1.83070976e-47  0.00000000e+00 -3.89171813e-46  0.00000000e+00  3.68673945e-46  0.00000000e+00  3.29575734e-47  0.00000000e+00 -5.20599484e-48  0.00000000e+00 -1.11014053e-47  0.00000000e+00  1.84757385e-45  0.00000000e+00 -2.11863523e-45  0.00000000e+00  2.76342281e-46  0.00000000e+00 -5.86793682e-50  0.00000000e+00  4.54755562e-45  0.00000000e+00 -4.55702960e-45  0.00000000e+00 -2.24438297e-46  0.00000000e+00  2.32409537e-46  0.00000000e+00 -3.92229974e-48  0.00000000e+00  5.05456139e-45  0.00000000e+00 -5.05853503e-45  0.00000000e+00 -1.93582212e-46  0.00000000e+00  5.86413369e-44  0.00000000e+00 -5.41741892e-44  0.00000000e+00 -4.26735034e-45  0.00000000e+00 -3.42227018e-48  0.00000000e+00  5.47472167e-44  0.00000000e+00 -5.47506304e-44  0.00000000e+00  3.13258378e-45  0.00000000e+00 -3.12612695e-45  0.00000000e+00  2.34063661e-49  0.00000000e+00  4.60589494e-44  0.00000000e+00 -4.60615762e-44  0.00000000e+00  1.95537792e-45  0.00000000e+00  1.43036107e-43  0.00000000e+00 -1.44984259e-43  0.00000000e+00 -2.22635805e-48  0.00000000e+00  3.54358216e-44  0.00000000e+00 -3.54354244e-44  0.00000000e+00  1.23344710e-43  0.00000000e+00 -1.22353383e-43  0.00000000e+00 -9.88041687e-46  0.00000000e+00  2.57979924e-44  0.00000000e+00 -2.57979015e-44  0.00000000e+00 -1.75141853e-48  0.00000000e+00  1.01177277e-43  0.00000000e+00 -1.00943978e-43  0.00000000e+00  1.64939318e-44  0.00000000e+00 -1.67248477e-44  0.00000000e+00 -1.51802579e-48  0.00000000e+00  7.80920259e-44  0.00000000e+00 -7.81152036e-44  0.00000000e+00  1.11233695e-44  0.00000000e+00 -1.11009452e-44  0.00000000e+00 -9.18689337e-50  0.00000000e+00  6.22876292e-44  0.00000000e+00 -6.23320948e-44  0.00000000e+00  8.13140193e-44  0.00000000e+00 -7.33283242e-44  0.00000000e+00 -7.94287995e-45  0.00000000e+00  4.97809765e-44  0.00000000e+00 -4.97811392e-44  0.00000000e+00 -4.19254071e-47  0.00000000e+00  7.32525206e-44  0.00000000e+00 -6.84557696e-44  0.00000000e+00 -4.75693467e-45  0.00000000e+00  3.61104002e-44  0.00000000e+00 -3.61113494e-44  0.00000000e+00 -3.34929037e-47  0.00000000e+00  6.63660490e-44  0.00000000e+00 -6.36938682e-44  0.00000000e+00  2.22181804e-44  0.00000000e+00 -2.48592095e-44  0.00000000e+00 -2.68351556e-47  0.00000000e+00  5.77148764e-44  0.00000000e+00 -5.62528981e-44  0.00000000e+00  3.58570834e-44  0.00000000e+00 -2.06987043e-44  0.00000000e+00 -1.65960586e-44  0.00000000e+00  4.78482539e-44  0.00000000e+00 -4.78484824e-44  0.00000000e+00  7.51757768e-46  0.00000000e+00  3.80205727e-44  0.00000000e+00 -2.79667984e-44  0.00000000e+00 -1.08079184e-44  0.00000000e+00  3.82789871e-44  0.00000000e+00 -3.82796575e-44  0.00000000e+00  2.83883878e-46  0.00000000e+00  3.87986224e-44  0.00000000e+00 -3.23435395e-44  0.00000000e+00 -6.74117780e-45  0.00000000e+00  2.92313943e-44  0.00000000e+00 -2.92315773e-44  0.00000000e+00  6.41966308e-47  0.00000000e+00  3.68769440e-44  0.00000000e+00 -3.25576203e-44  0.00000000e+00  1.03922461e-44  0.00000000e+00  7.70292064e-45  0.00000000e+00 -2.24806543e-44  0.00000000e+00  3.36782022e-44  0.00000000e+00 -3.36712809e-44  0.00000000e+00  3.03001557e-45  0.00000000e+00  1.27213913e-44  0.00000000e+00  1.74741886e-45  0.00000000e+00 -1.75079762e-44  0.00000000e+00  3.00238932e-44  0.00000000e+00 -3.00307966e-44  0.00000000e+00  1.75553865e-45  0.00000000e+00  1.50298926e-44  0.00000000e+00 -4.95038574e-45  0.00000000e+00 -1.18303412e-44  0.00000000e+00  2.56605626e-44  0.00000000e+00 -2.56667301e-44  0.00000000e+00  9.42720850e-46  0.00000000e+00  1.83000856e-44  0.00000000e+00 -1.13394789e-44  0.00000000e+00  1.34642336e-44  0.00000000e+00 -2.13682414e-44  0.00000000e+00  1.94441808e-44  0.00000000e+00 -1.88070548e-44  0.00000000e+00  5.37720340e-45  0.00000000e+00  2.22527685e-46  0.00000000e+00  1.18498229e-44  0.00000000e+00 -2.50262577e-47  0.00000000e+00]'


a = '[           -inf           +nanj -7.17444012e-04+0.00000000e+00j            -inf           +nanj -1.66272925e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-6.32908078e-04j            -inf           +nanj  0.00000000e+00+5.95966399e-05j            -inf           +nanj -2.95581624e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.11092886e-03j             inf           +nanj  0.00000000e+00-5.42182515e-04j            -inf           +nanj -5.03622004e-06+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.61832518e-03j             inf           +nanj  0.00000000e+00-5.36459692e-04j            -inf           +nanj -2.30572513e-04+0.00000000e+00j             inf           +nanj  4.10712405e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-6.85690220e-04j             inf           +nanj  6.57135531e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.12692460e-03j             inf           +nanj  0.00000000e+00-1.69327773e-04j            -inf           +nanj -1.46607888e-03+0.00000000e+00j             inf           +nanj  2.92950056e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.90588838e-03j             inf           +nanj  0.00000000e+00-1.17871930e-03j             inf           +nanj  7.04704674e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-3.31376840e-03j            -inf           +nanj -1.13719854e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.42684849e-02j             inf           +nanj  5.11840388e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-5.50210446e-03j             inf           +nanj  0.00000000e+00-1.79225407e-04j            -inf           +nanj -1.01513612e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.68142498e-03j             inf           +nanj  4.23747199e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-5.79289086e-03j             inf           +nanj  0.00000000e+00-1.93419290e-04j            -inf           +nanj -5.35715604e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.03308559e-02j             inf           +nanj  0.00000000e+00-5.33476027e-03j             inf           +nanj  0.00000000e+00-1.72655807e-03j             inf           +nanj  1.24672697e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.89448146e-02j             inf           +nanj  4.71843720e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-4.58076187e-03j             inf           +nanj  6.53853016e-03+0.00000000e+00j             inf           +nanj  1.48279997e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.73801007e-02j             inf           +nanj  6.71588800e-05+0.00000000e+00j             inf           +nanj  4.26207178e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-3.08197405e-02j             inf           +nanj  0.00000000e+00-2.38218172e-03j             inf           +nanj  1.79251394e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.52500566e-02j             inf           +nanj  9.69931180e-06+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.85428758e-02j             inf           +nanj  0.00000000e+00-2.66028609e-03j             inf           +nanj  2.28633996e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.30194921e-02j             inf           +nanj  0.00000000e+00-2.78040448e-03j             inf           +nanj  1.11225459e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.57790794e-02j             inf           +nanj  1.74537741e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.04933382e-02j             inf           +nanj  0.00000000e+00-3.02801039e-03j             inf           +nanj  1.29108243e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.26216247e-02j             inf           +nanj  1.64265336e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-8.55910805e-03j             inf           +nanj  0.00000000e+00-1.17023142e-02j             inf           +nanj  1.41822143e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.02016952e-02j            -inf           +nanj -2.05077690e-07+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.42975608e-02j             inf           +nanj  0.00000000e+00-7.24972297e-03j             inf           +nanj  1.52654615e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.80690387e-02j             inf           +nanj  0.00000000e+00-5.23987956e-04j             inf           +nanj  1.31824050e-09+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.26545041e-02j             inf           +nanj  0.00000000e+00-5.62757087e-03j             inf           +nanj  1.69562835e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.53930998e-02j             inf           +nanj  0.00000000e+00-5.43491146e-04j             inf           +nanj  1.68941828e-06+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.12798879e-02j             inf           +nanj  1.45352288e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.27766903e-02j             inf           +nanj  0.00000000e+00-5.70609112e-04j             inf           +nanj  3.26549782e-06+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.96936065e-02j             inf           +nanj  6.28870137e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.09946643e-02j             inf           +nanj  0.00000000e+00-1.04454735e-02j             inf           +nanj  1.25749376e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.77124010e-02j             inf           +nanj  9.96484690e-05+0.00000000e+00j             inf           +nanj  3.25976798e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.87765400e-02j             inf           +nanj  0.00000000e+00-8.43659301e-03j             inf           +nanj  1.29777124e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.58441619e-02j             inf           +nanj  1.47478481e-04+0.00000000e+00j             inf           +nanj  1.29234173e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.75981385e-02j             inf           +nanj  0.00000000e+00-6.67170680e-03j             inf           +nanj  1.34872273e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.38478893e-02j             inf           +nanj  2.85951679e-04+0.00000000e+00j             inf           +nanj  3.89811277e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.65788237e-02j             inf           +nanj  3.50188226e-03+0.00000000e+00j             inf           +nanj  1.36486825e-02+0.00000000e+00j            inf           +nanj  0.00000000e+00-1.21464722e-02j             inf           +nanj  1.16171787e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.48626311e-02j             inf           +nanj  4.89047669e-05+0.00000000e+00j             inf           +nanj  2.19764224e-03+0.00000000e+00j             inf           +nanj  3.05544267e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.07214325e-02j             inf           +nanj  1.12628947e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.40303756e-02j             inf           +nanj  3.32457053e-05+0.00000000e+00j             inf           +nanj  1.17005804e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.93239164e-02j             inf           +nanj  0.00000000e+00-8.81737959e-03j             inf           +nanj  1.10720482e-05+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.29714554e-02j             inf           +nanj  4.15118049e-05+0.00000000e+00j             inf           +nanj  5.73310752e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.46340390e-02j             inf           +nanj  5.52231095e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.18366168e-02j             inf           +nanj  8.74573093e-06+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.14816087e-02j             inf           +nanj  7.19677375e-04+0.00000000e+00j             inf           +nanj  3.27053741e-02+0.00000000e+00j             inf           +nanj  4.64723959e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.92792285e-01j             inf           +nanj]'


b = '[           -inf           +nanj -2.85959464e-04+0.00000000e+00j            -inf           +nanj  5.75802732e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00+4.22123770e-05j            -inf           +nanj  0.00000000e+00+5.37785330e-04j            -inf           +nanj  5.74441550e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-6.61055357e-04j             inf           +nanj  0.00000000e+00+6.32519588e-06j            -inf           +nanj  6.83857172e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.32721886e-03j             inf           +nanj  0.00000000e+00+1.83746884e-04j            -inf           +nanj  3.43371505e-04+0.00000000e+00j             inf           +nanj  8.90551871e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-6.24905260e-05j             inf           +nanj  6.58347414e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-7.80708913e-04j             inf           +nanj  0.00000000e+00+1.42694710e-03j            -inf           +nanj  8.75890825e-05+0.00000000e+00j             inf           +nanj  7.57480951e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.72126409e-03j             inf           +nanj  0.00000000e+00-9.09115530e-04j             inf           +nanj  3.54853705e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00+4.10706775e-04j            -inf           +nanj  2.07920307e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.42488839e-02j             inf           +nanj  5.50783279e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-3.97747201e-05j             inf           +nanj  0.00000000e+00+1.03300583e-03j            -inf           +nanj  2.18434338e-04+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.52117153e-03j             inf           +nanj  5.79516745e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-3.78372991e-05j             inf           +nanj  0.00000000e+00+9.32402661e-04j            -inf           +nanj  1.95411420e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.49726216e-03j             inf           +nanj  0.00000000e+00-4.88944074e-05j             inf           +nanj  0.00000000e+00-1.57686761e-03j             inf           +nanj  1.89454052e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.12863877e-05j             inf           +nanj  4.57603854e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-5.65775091e-05j             inf           +nanj  6.57767766e-03+0.00000000e+00j             inf           +nanj  1.73805963e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.38372377e-05j             inf           +nanj  3.64525779e-03+0.00000000e+00j             inf           +nanj  3.10289131e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-9.33494938e-06j             inf           +nanj  0.00000000e+00-2.26144636e-03j             inf           +nanj  1.52499711e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.80959908e-05j             inf           +nanj  2.84279442e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.39060589e-04j             inf           +nanj  0.00000000e+00-1.16827781e-04j             inf           +nanj  1.30194691e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.29092440e-05j             inf           +nanj  0.00000000e+00-2.67331866e-03j             inf           +nanj  2.57493410e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-7.05523103e-05j             inf           +nanj  1.05665365e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.88480093e-05j             inf           +nanj  0.00000000e+00-2.92831466e-03j             inf           +nanj  2.26249815e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-6.19863870e-06j             inf           +nanj  8.55047628e-03+0.00000000e+00j             inf           +nanj  0.00000000e+00-3.36648058e-05j             inf           +nanj  0.00000000e+00-1.16777885e-02j             inf           +nanj  2.02089047e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00+2.34231483e-07j            -inf           +nanj  2.30736257e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.38602248e-03j             inf           +nanj  0.00000000e+00-3.82166527e-05j             inf           +nanj  1.80690683e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.52064393e-05j             inf           +nanj  0.00000000e+00-5.51019912e-08j             inf           +nanj  2.19002111e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.48347263e-03j             inf           +nanj  0.00000000e+00-4.67178907e-05j             inf           +nanj  1.53933022e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.65518680e-05j             inf           +nanj  0.00000000e+00-7.52027582e-05j             inf           +nanj  2.08470768e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-8.58474309e-04j             inf           +nanj  1.35147427e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.87476615e-05j             inf           +nanj  0.00000000e+00-1.51440218e-04j             inf           +nanj  1.94425768e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-5.02083561e-04j             inf           +nanj  1.59511881e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-9.35317437e-03j             inf           +nanj  0.00000000e+00-2.13518940e-05j             inf           +nanj  1.77124433e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.24903759e-05j             inf           +nanj  2.31823285e-03+0.00000000e+00j             inf           +nanj  1.61037720e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-5.24465290e-03j             inf           +nanj  0.00000000e+00-2.44234682e-05j             inf           +nanj  1.58443006e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.27003298e-05j             inf           +nanj  1.51082845e-03+0.00000000e+00j             inf           +nanj  1.60676496e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-3.04586795e-03j             inf           +nanj  0.00000000e+00-2.80853312e-05j             inf           +nanj  1.38479326e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.34005506e-05j             inf           +nanj  9.34277184e-04+0.00000000e+00j             inf           +nanj  1.55776728e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.97847207e-03j             inf           +nanj  1.17506998e-02+0.00000000e+00j             inf           +nanj  2.07504214e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.42190521e-05j             inf           +nanj  1.48611038e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.46704624e-05j             inf           +nanj  4.50299918e-03+0.00000000e+00j             inf           +nanj  1.13241430e-02+0.00000000e+00j             inf           +nanj  3.39369087e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.47491011e-05j             inf           +nanj  1.40319886e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-8.03817252e-06j             inf           +nanj  3.42357737e-03+0.00000000e+00j             inf           +nanj  1.10901324e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-1.36307155e-02j             inf           +nanj  0.00000000e+00-1.63065625e-05j             inf           +nanj  1.29730141e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-7.95569089e-06j             inf           +nanj  2.52595199e-03+0.00000000e+00j             inf           +nanj  1.15195150e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-6.01748655e-03j             inf           +nanj  1.49114940e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-8.34269846e-06j             inf           +nanj  1.12919335e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-3.84818274e-04j             inf           +nanj  6.65323577e-03+0.00000000e+00j             inf           +nanj  3.39124345e-02+0.00000000e+00j             inf           +nanj  1.34555562e-02+0.00000000e+00j             inf           +nanj  0.00000000e+00-2.92387491e-01j             inf           +nanj]'

In [39]:
test = b.replace('[', '').replace(']', '').replace('j', 'j,').split(',')
np.array([test[i].replace(' ','') for i in range(len(test)-1)], dtype=np.complex)

array([           -inf           +nanj, -2.85959464e-04+0.00000000e+00j,
                  -inf           +nanj,  5.75802732e-04+0.00000000e+00j,
                   inf           +nanj,  0.00000000e+00+4.22123770e-05j,
                  -inf           +nanj,  0.00000000e+00+5.37785330e-04j,
                  -inf           +nanj,  5.74441550e-04+0.00000000e+00j,
                   inf           +nanj,  0.00000000e+00-6.61055357e-04j,
                   inf           +nanj,  0.00000000e+00+6.32519588e-06j,
                  -inf           +nanj,  6.83857172e-04+0.00000000e+00j,
                   inf           +nanj,  0.00000000e+00-1.32721886e-03j,
                   inf           +nanj,  0.00000000e+00+1.83746884e-04j,
                  -inf           +nanj,  3.43371505e-04+0.00000000e+00j,
                   inf           +nanj,  8.90551871e-04+0.00000000e+00j,
                   inf           +nanj,  0.00000000e+00-6.24905260e-05j,
                   inf           +nanj,  6.58347414

In [81]:
import numpy as np

ds = 400 * 3.086*10**18 * 0.18046111240715157

# test_intensity = 0
# test_epsilon = np.array(epsilon.replace('[', '').replace(']', '').split()).astype(np.float)
# test_depsilon = np.array(depsilon.replace('[', '').replace(']', '').split()).astype(np.float)
# test_kappa = np.array(kappa.replace('[', '').replace(']', '').split()).astype(np.float)
# test_dkappa = np.array(dkappa.replace('[', '').replace(']', '').split()).astype(np.float)
# test_a = a.replace('[', '').replace(']', '').replace('j', 'j,').split(',')
# test_a = np.array([test_a[i].replace(' ','') for i in range(len(test_a)-1)], dtype=np.complex)
# test_b = b.replace('[', '').replace(']', '').replace('j', 'j,').split(',')
# test_b = np.array([test_b[i].replace(' ','') for i in range(len(test_b)-1)], dtype=np.complex)
test_intensity = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\I.npy')
test_epsilon = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\e.npy')
test_depsilon = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\de.npy')
test_kappa = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\k.npy')
test_dkappa = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\dk.npy')
test_a = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\a.npy')
test_b = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\b.npy')

In [82]:
from scipy import interpolate

GRIDPATH = r'c:\users\cyani\KOSMA-tau^3\kosmatau3d\grid'

def eTildeRealData(file='\Ereal.dat'):
    ereal = np.genfromtxt(GRIDPATH+file, names=['x', 'Ereal'])
    return (ereal['x'], ereal['Ereal'])


def eTildeImaginaryData(file='\Eimag.dat'):
    eimaginary = np.genfromtxt(GRIDPATH+file, names=['x', 'Eimaginary'])
    return (eimaginary['x'], eimaginary['Eimaginary'])

eTildeRealObs = eTildeRealData()
eTildeImaginaryObs = eTildeImaginaryData()

eTildeReal = interpolate.interp1d(eTildeRealObs[0], eTildeRealObs[1], kind='linear')
eTildeImaginary = interpolate.interp1d(eTildeImaginaryObs[0], eTildeImaginaryObs[1], kind='linear')

def Ereal(x, test=False):
    Er = 0
    if test is True:
      il = x < 0.01
      ig = x > 8.0
      ib = ~(ig | il)
      if il:
          Er = (2*x/np.sqrt(np.pi)).astype(np.complex_)
      elif ig:
          Er = (1/(np.sqrt(np.pi) * x)).astype(np.complex_)
      else:
          Er = eTildeReal(x.real).astype(np.complex_)
    elif test is False:
      if x < 0.01:
          Er = (2*x/np.sqrt(np.pi)).astype(np.complex_)
      elif x > 8.0:
          Er = (1/(np.sqrt(np.pi) * x)).astype(np.complex_)
      else:
          Er = eTildeReal(x.real).astype(np.complex_)
    return Er

def Eimag(x, test=False):
    Ei = 0
    if test is True:
      im = x == abs(x)*1j
      il = (abs(x) < 0.01) & ~im
      ig = (abs(x) > 8.0) & ~im
      ib = ~(ig | il | im)
      if im:
          # MASER case: treat with linear approximation
          Ei = 1 + 2*abs(x)/np.sqrt(np.pi)
      elif il:
          Ei = (1 - 2*abs(x)/np.sqrt(np.pi)).astype(np.complex_)
      elif ig:
          Ei = (1/(np.sqrt(np.pi) * abs(x))).astype(np.complex_)
      else:
          Ei = eTildeImaginary(abs(x)).astype(np.complex_)
    if test is False:
      if x == (1j*abs(x)):
          # MASER case: treat with linear approximation
          Ei = 1 + 2*abs(x)/np.sqrt(np.pi)
      elif (x == (-1j*abs(x))) and (abs(x) < 0.01):
          Ei = (1 - 2*abs(x)/np.sqrt(np.pi)).astype(np.complex_)
      elif (x == (-1j*abs(x))) and (abs(x) > 8.0):
          Ei = (1/(np.sqrt(np.pi) * abs(x))).astype(np.complex_)
      else:
          Ei = eTildeImaginary(abs(x)).astype(np.complex_)
    return Ei

def rt_basic(i, spe, intensity):
  return test_epsilon[i, 0, spe]*ds + 0.5*test_depsilon[i, 0, spe]*ds**2 + intensity

def rt_simple(i, spe, intensity):
  return ((test_epsilon[i, 0, spe]*test_kappa[i, 0, spe] + test_depsilon[i, 0, spe]*(test_kappa[i, 0, spe]*ds-1))/test_kappa[i, 0, spe]**2) - np.exp(-test_kappa[i, 0, spe]*ds)*((test_epsilon[i, 0, spe]*test_kappa[i, 0, spe] - test_depsilon[i, 0, spe])/test_kappa[i, 0, spe]**2) + np.exp(-test_kappa[i, 0, spe]*ds)*intensity

def rt_complex1(i, spe, intensity, test=False):
  return (test_depsilon[i, 0, spe]/test_dkappa[i, 0, spe]*(1-np.exp(-test_kappa[i, 0, spe]*ds - 0.5*test_dkappa[i, 0, spe]*ds**2)) - (test_epsilon[i, 0, spe]*test_dkappa[i, 0, spe] - test_kappa[i, 0, spe]*test_depsilon[i, 0, spe])/test_dkappa[i, 0, spe]*np.sqrt(np.pi/2/np.abs(test_dkappa[i, 0, spe]))*(np.exp(test_a[i, 0, spe]**2-test_b[i, 0, spe]**2)*Ereal(test_a[i, 0, spe], test=test) - Ereal(test_b[i, 0, spe], test=test)) + intensity*np.exp(-test_kappa[i, 0, spe]*ds - 0.5*test_dkappa[i, 0, spe]*ds**2)).real

def rt_complex2(i, spe, intensity, test=False):
  return (test_depsilon[i, 0, spe]/test_dkappa[i, 0, spe]*(1-np.exp(-test_kappa[i, 0, spe]*ds - 0.5*test_dkappa[i, 0, spe]*ds**2)) - (test_epsilon[i, 0, spe]*test_dkappa[i, 0, spe] - test_kappa[i, 0, spe]*test_depsilon[i, 0, spe])/test_dkappa[i, 0, spe]*np.sqrt(np.pi/2/np.abs(test_dkappa[i, 0, spe]))*(np.exp(test_a[i, 0, spe]**2-test_b[i, 0, spe]**2)*Eimag(test_a[i, 0, spe], test=test) - Eimag(test_b[i, 0, spe], test=test)) + intensity*np.exp(-test_kappa[i, 0, spe]*ds - 0.5*test_dkappa[i, 0, spe]*ds**2)).real


In [83]:
intensity = 0
spe = 0
test = False
PRINT = False

for i in range(test_depsilon[:, 0, spe].size):
  
  k0 = (test_dkappa[i, 0, spe] == 0) and (abs(test_kappa[i, 0, spe]*ds) < 10**-10)
  kg = (test_dkappa[i, 0, spe] == 0) or (np.abs(test_kappa[i, 0, spe]) > np.abs(10**3*test_dkappa[i, 0, spe]*ds))
  keg = test_dkappa[i, 0, spe] > 0
  kel = test_dkappa[i, 0, spe] < 0
  
  if k0:
    if PRINT is True:
      print(1)
      print('Intensity:', intensity)
      print('epsilon:', test_epsilon[i], test_depsilon[i])
      print('kappa:', test_kappa[i], test_dkappa[i])
    intensity = rt_basic(i, spe, intensity)
  elif kg:
    if PRINT is True:
      print(2)
      print('Intensity:', intensity)
      print('epsilon:', test_epsilon[i], test_depsilon[i])
      print('kappa:', test_kappa[i], test_dkappa[i])
      print(10**3*test_dkappa[i]*ds)
    intensity = rt_simple(i, spe, intensity)
  else:
    if keg:
      if PRINT is True:
        print(3)
        print('Intensity:', intensity)
        print('epsilon:', test_epsilon[i], test_depsilon[i])
        print('kappa:', test_kappa[i], test_dkappa[i])
      intensity = rt_complex1(i, spe, intensity, test=test)
    elif kel:
      if PRINT is True:
        print(4)
        print('Intensity:', intensity)
        print('epsilon:', test_epsilon[i], test_depsilon[i])
        print('kappa:', test_kappa[i], test_dkappa[i])
      intensity = rt_complex2(i, spe, intensity, test=test)
    else:
      print('Error!')

print('Old intensity:', test_intensity[spe])
print('Test intensity:', intensity)

Old intensity: -1.004523263551004
Test intensity: 6.3144540199712305


In [62]:
test_intensity

array([-0.0002081, -0.0002081, -0.0002081, -0.0002081, -0.0002081,
       -0.0002081, -0.0002081, -0.0002081, -0.0002081, -0.0002081,
       -0.0002081, -0.0002081, -0.0002081, -0.0002081, -0.0002081,
       -0.0002081, -0.0002081, -0.0002081])

In [80]:
test_kappa[:,0,0]

array([-1.37672091e-26, -1.37672091e-26, -6.36102238e-27, -6.36102238e-27,
       -6.36102238e-27, -6.36102238e-27, -4.06058943e-27, -4.06058943e-27,
       -2.13553813e-26, -2.13553813e-26, -2.13553813e-26, -2.13553813e-26,
       -6.34573109e-27, -6.34573109e-27, -2.04839964e-26, -2.04839964e-26,
       -2.04839964e-26, -2.04839964e-26, -2.03593034e-26, -2.03593034e-26,
       -2.03593034e-26, -2.03593034e-26, -3.51259108e-26, -3.51259108e-26,
       -3.51259108e-26, -3.51259108e-26, -1.52534557e-26, -1.52534557e-26,
       -2.63626104e-26, -2.63626104e-26, -2.63626104e-26, -2.63626104e-26,
       -4.77623688e-26, -4.77623688e-26, -4.77623688e-26, -4.77623688e-26,
       -3.31521894e-26, -3.31521894e-26, -4.37968419e-26, -4.37968419e-26,
       -4.37968419e-26, -4.37968419e-26, -6.47119821e-26, -6.47119821e-26,
       -6.47119821e-26, -6.47119821e-26, -5.06740090e-26, -5.06740090e-26,
       -5.99023963e-26, -5.99023963e-26, -5.99023963e-26, -5.99023963e-26,
       -8.42908725e-26, -